# Importing libraries and loading the MNIST dataset

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_data, mnist_info = tfds.load(name='mnist', with_info= True, as_supervised= True)

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

# Preprocessing the data:

In [4]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples 
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label 
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000 

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_dataset = shuffled_train_and_validation_data.take(num_validation_samples)
train_dataset = shuffled_train_and_validation_data.skip(num_validation_samples)
test_dataset = scaled_test_data.shuffle(100)

BATCH_SIZE = 100

train_data = train_dataset.batch(BATCH_SIZE)
validation_data = validation_dataset.batch(num_validation_samples)
test_data = test_dataset.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Outlining the model:

In [5]:
input_size = 784 
output_size = 10
hidden_layer_size = 350

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation= 'tanh'),
                            tf.keras.layers.Dense(output_size, activation= 'softmax')
                            ])

# Choosing optimizer and Loss function:

In [38]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)

model.compile(optimizer = custom_optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Training:

In [39]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), validation_steps = 10, verbose = 2)

Epoch 1/5
540/540 - 5s - loss: 0.0156 - accuracy: 0.9965 - val_loss: 0.0234 - val_accuracy: 0.9940
Epoch 2/5
540/540 - 4s - loss: 0.0126 - accuracy: 0.9975 - val_loss: 0.0199 - val_accuracy: 0.9947
Epoch 3/5
540/540 - 4s - loss: 0.0104 - accuracy: 0.9980 - val_loss: 0.0162 - val_accuracy: 0.9963
Epoch 4/5
540/540 - 4s - loss: 0.0085 - accuracy: 0.9988 - val_loss: 0.0125 - val_accuracy: 0.9967
Epoch 5/5
540/540 - 4s - loss: 0.0065 - accuracy: 0.9989 - val_loss: 0.0102 - val_accuracy: 0.9970


# Testing: 

In [40]:
test_loss, test_accuracy = model.evaluate(test_data)
test_loss, test_accuracy

      1/Unknown - 1s 957ms/step - loss: 0.0602 - accuracy: 0.9820

(0.0601995587348938, 0.982)